
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# DEMO - Real-time Deployment with Model Serving

In this demo, we will serve the model stored in the model registry using Mosaic Model Serving. We will utilize the Agent Framework to serve the models. When models are served with the Agent Framework, an app called Review App is automatically deployed alongside the model, allowing you to interact with the model and gather human feedback on its responses.

**Learning Objectives:**

*By the end of this demo, you will be able to:*

- Deploy a model using the Agent Framework.
- Use the Review App to interact with the model and collect human feedback.

## REQUIRED - SELECT CLASSIC COMPUTE
Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default.

Follow these steps to select the classic compute cluster:
1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.
   
   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.

## Requirements

Please review the following requirements before starting the lesson:

* To run this notebook, you need to use one of the following Databricks runtime(s): **16.2.x-cpu-ml-scala2.12**



## Classroom Setup

Install required libraries.

In [0]:
%pip install -qq -U databricks-sdk langchain-databricks databricks-vectorsearch databricks-agents
dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


**Other Conventions:**

Throughout this demo, we'll refer to the object `DA`. This object, provided by Databricks Academy, contains variables such as your username, catalog name, schema name, working directory, and dataset locations. Run the code block below to view these details:

In [0]:
%run ../Includes/Classroom-Setup-03


The examples and models presented in this course are intended solely for demonstration and educational purposes.
 Please note that the models and prompt examples may sometimes contain offensive, inaccurate, biased, or harmful content.


In [0]:
model_name = f"{DA.catalog_name}.{DA.schema_name}.getstarted_genai_rag_demo"

print(f"= Variables that you will need for this demo = \n")
print(f"Catalog Name                : {DA.catalog_name}\n")
print(f"Schema Name                 : {DA.schema_name}\n")
print(f"RAG Registered Model        : {model_name} \n")

= Variables that you will need for this demo = 

Catalog Name                : dbacademy

Schema Name                 : labuser10813094_1751496013

RAG Registered Model        : dbacademy.labuser10813094_1751496013.getstarted_genai_rag_demo 



## Serve the Model with Agent Framework

**🚨 Note:** This step is intended for the course instructor only. If you are using your own environment, feel free to comment out the cells and run them to deploy the model and access the Review App.


In [0]:
import time
import mlflow
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointStateReady, EndpointStateConfigUpdate
from databricks import agents

model_name = f"{DA.catalog_name}.{DA.schema_name}.getstarted_genai_rag_demo"
# Deploy the model with the agent framework
deployment_info = agents.deploy(
    model_name, 
    model_version=1,
    scale_to_zero=True)

# Wait for the Review App and deployed model to be ready
w = WorkspaceClient()
print("\nWaiting for endpoint to deploy.  This can take 15 - 20 minutes.", end="")

while ((w.serving_endpoints.get(deployment_info.endpoint_name).state.ready == EndpointStateReady.NOT_READY) or (w.serving_endpoints.get(deployment_info.endpoint_name).state.config_update == EndpointStateConfigUpdate.IN_PROGRESS)):
    print(".", end="")
    time.sleep(30)

print("\nThe endpoint is ready!", end="")

/local_disk0/.ephemeral_nfs/envs/pythonEnv-f3767cc7-dd42-4422-9b44-969566286459/lib/python3.12/site-packages/databricks/agents/utils/mlflow_utils.py:149: FutureWarning: ``mlflow.models.rag_signatures.ChatCompletionRequest`` is deprecated. This method will be removed in a future release. Use ``mlflow.types.llm.ChatCompletionRequest`` instead.
  ChatCompletionRequest()
/local_disk0/.ephemeral_nfs/envs/pythonEnv-f3767cc7-dd42-4422-9b44-969566286459/lib/python3.12/site-packages/mlflow/models/rag_signatures.py:26: FutureWarning: ``mlflow.models.rag_signatures.Message`` is deprecated. This method will be removed in a future release. Use ``mlflow.types.llm.ChatMessage`` instead.
  messages: list[Message] = field(default_factory=lambda: [Message()])
/local_disk0/.ephemeral_nfs/envs/pythonEnv-f3767cc7-dd42-4422-9b44-969566286459/lib/python3.12/site-packages/databricks/agents/utils/mlflow_utils.py:152: FutureWarning: ``mlflow.models.rag_signatures.SplitChatMessagesRequest`` is deprecated. This m

2025/07/02 23:40:00 INFO mlflow.pyfunc: Validating input example against model signature


Uploading artifacts:   0%|          | 0/12 [00:00<?, ?it/s]

Successfully registered model 'dbacademy.labuser10813094_1751496013.feedback'.


Uploading artifacts:   0%|          | 0/12 [00:00<?, ?it/s]

Created version '1' of model 'dbacademy.labuser10813094_1751496013.feedback'.



    Deployment of dbacademy.labuser10813094_1751496013.getstarted_genai_rag_demo version 1 initiated.  This can take up to 15 minutes and the Review App & Query Endpoint will not work until this deployment finishes.

    View status: https://dbc-be792efb-7a0c.cloud.databricks.com/ml/endpoints/agents_dbacademy-labuser10813094_1751496013-getstarted_genai_ra
    Review App: https://dbc-be792efb-7a0c.cloud.databricks.com/ml/review-v2/5c959b816cac426ea64c6d87a6b0af84/chat

Waiting for endpoint to deploy.  This can take 15 - 20 minutes.................
The endpoint is ready!

## Collect Human Feedback via Databricks Review App

The Databricks Review App stages the LLM in an environment where expert stakeholders can engage with it—allowing for conversations, questions, and more. This setup enables the collection of valuable feedback on your application, ensuring the quality and safety of its responses.

**Stakeholders can interact with the application bot and provide feedback on these interactions. They can also offer feedback on historical logs, curated traces, or agent outputs.**


In [0]:
print(f"Endpoint URL    : {deployment_info.endpoint_url}")
print(f"Review App URL  : {deployment_info.review_app_url}")

Endpoint URL    : https://dbc-be792efb-7a0c.cloud.databricks.com/ml/endpoints/agents_dbacademy-labuser10813094_1751496013-getstarted_genai_ra
Review App URL  : https://dbc-be792efb-7a0c.cloud.databricks.com/ml/review-v2/5c959b816cac426ea64c6d87a6b0af84/chat


## Summary

In this demo, we first deployed the registered model using the Agent Framework. Then, we interacted with the deployed model through the Review App. Additionally, we demonstrated how the Review App can be used to collect human feedback, which can later be analyzed or used to improve the model.


&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>